In [1]:
#import re
#import urllib.request

In [99]:
#response = urllib.request.urlopen('https://aip.com.au/pricing/new-south-walesact-retail-petrol-prices')
#html = response.read()
#text = html.decode()

#print (text)

In [146]:
from selenium import webdriver
import re # Importing regex
import datetime
import os
import csv

In [147]:
# Driver is a necessary program that is used to emulate a web browser activity allowing us to view dynamic content
# As the site that we are targeting uses dynamic content, we can only view it via this
# The path to the web driver is relative to the program, hence just place this program and the webdriver in the same folder
path = r'.\Chromedriver'
driver = webdriver.Chrome(executable_path = path)

In [148]:
# Executing the driver with the web address we're targeting
driver.get('https://aip.com.au/pricing/new-south-walesact-retail-petrol-prices')

In [149]:

def split_string(fstring):
    # Return is string is empty
    if len(fstring) <= 0:
        return []
    
    # Check if current char is not a white space
    if fstring[:1] != ' ':
        # Current value found, get word remainder
        value = split_word(fstring)
        
        # Return array of found value and remainder
        return [value[0]] + split_string(value[1])
        
    else:
        # No current value, continue to next
        return split_string(fstring[1:])
    
    # If current char is whitespace but next is a letter, then complete sentence
    #elif fstring[1:][:1].isalpha():
    

def split_word(fstring):
    # Check if we've got chars left
    if len(fstring) <= 0:
        return '', ''
    
    # Check if we haven't reached the end of the word
    if (fstring[:1] != ' ') | ((len(fstring) >= 2) & (fstring[1:2].isalpha())):
        value = split_word(fstring[1:])
        return fstring[:1] + value[0] , value[1]
    
    else:
        return '', fstring
    
    return '', ''



In [150]:
def get_Canberra(flist):
    for element in flist:
        if 'Canberra' in element:
            return element

In [151]:
#test = "Week ending Sunday, 13th September 2020"
#test = "13th September 2020"

#def get_date(fstring):
    #Week ending Sunday, 13th September 2020 Date example
    #ddxx
    #regex = re.search(r"\b[1-3]?[0-9]{1}((th)|(rd)|(st)) [A-Z][a-z]+ [0-9]{4}", fstring)
#    regex = re.search(r"\b[1-3]?[0-9]{1}.* [A-Z][a-z]+ [0-9]{4}", fstring)
#    return regex.group().sub("","((th)|(rd)|(st))") if regex else "None"

#%d %B %Y
#print (get_date(test))

In [156]:
# Driver.find_elements_by_css_selector returns an Array of elements that match the selector's criteria
# We have this in an array which is populated by a for loop which grabs the text values for the elements
#elements = [element.text.split(' ') for element in driver.find_elements_by_css_selector('tr')]
elements = [split_string(element.text) for element in driver.find_elements_by_css_selector('tr')]

# Want to get the current week
#current_week = " ".join(elements[0])

In [171]:
# Getting date of last Sunday which represents the end of the last week period in order to record accurate data
curr_day = datetime.date.today()
idx = (curr_day.weekday() + 1) % 7
# Calculation to get the last Sunday
curr_week = curr_day - datetime.timedelta(idx)
## I know it's ugly, but I am struggling to do better
curr_time = datetime.datetime(curr_week.year,curr_week.month,curr_week.day)
curr_timestamp = curr_time.timestamp()

In [172]:
targets = ['Canberra', 'Sydney', 'Batemans Bay', 'Cooma', 'Goulburn', 'Coffs Harbour', 'Wollongong', 'Yass']
# Will work on getting this single lined
current_prices = []
for element in elements:
    if element[0] in targets:
        current_prices = current_prices + [[curr_timestamp] + [curr_time.strftime("%Y-%m-%d")] + element]

print (current_prices)

[[1599919200.0, '2020-09-13', 'Sydney', '124.4', '-9.6', '-', '118.8', '129.7'], [1599919200.0, '2020-09-13', 'Canberra', '122.2', '-0.1', '-2.1', '122.0', '122.3'], [1599919200.0, '2020-09-13', 'Batemans Bay', '122.6', '0.1', '-1.8', '122.6', '122.6'], [1599919200.0, '2020-09-13', 'Coffs Harbour', '119.5', '-0.0', '-4.9', '119.4', '119.5'], [1599919200.0, '2020-09-13', 'Cooma', '123.5', '-0.1', '-0.8', '123.3', '123.7'], [1599919200.0, '2020-09-13', 'Goulburn', '119.0', '-0.6', '-5.4', '118.2', '119.6'], [1599919200.0, '2020-09-13', 'Wollongong', '123.9', '-2.4', '-0.5', '123.3', '124.7'], [1599919200.0, '2020-09-13', 'Yass', '120.0', '0.3', '-4.3', '119.5', '121.8']]


In [187]:
fileName = "fuel_data.csv"

if os.path.exists(fileName):
    with open(fileName) as fd:
        # Read CSV
        reader = csv.reader(fd, delimiter=',')
        # Skip header line
        next(reader)
        for element in reader:
            # Compare and skip if we have completed this timestamp before
            if float(element[0]) == curr_timestamp:
                # Return that we found that we have done this week
                return True
else:
    with open(fileName, 'w') as fd:
        writer = csv.writer(fd)
        writer.writerow(["Timestamp (Unix)","Date (YYYY-MM-DD)","Location", "Weekly Average", "Weekly Change", "Variation", "Weekly Low", "Weekly High"])
        

with open(fileName, 'a', newline='') as fd:
    writer = csv.writer(fd)
    for element in current_prices:
        writer.writerow(element)
    

Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes


In [131]:
print(datetime.datetime(curr_week.year,curr_week.month,curr_week.day).timestamp())

1599919200.0


In [ ]:
with open()

In [145]:
driver.close()